# 1.1 Setup a Folder:

---






In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive

In [ ]:
import os
if not os.path.isdir("CamVid"):
  os.makedirs("CamVid")
%cd CamVid

## 1.2 Donwloading DataSet:

---





In [ ]:
cd /content/drive/MyDrive

## 2.1 Exploring & Visualize The Data:

---






In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

imagesname=[string.split(".")[0] for string in (os.listdir("camvid_11/CamVid_11/train"))]
index=np.random.randint(len(imagesname))
Size=224

image=cv2.imread("camvid_11/CamVid_11/train/{}.png".format(imagesname[index]))
image=cv2.resize(image,(Size,Size))
image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

mask=cv2.imread("camvid_11/CamVid_11/train_labels/{}.png".format(imagesname[index]+"_L"))
mask=cv2.resize(mask,(Size,Size))
mask=cv2.cvtColor(mask,cv2.COLOR_BGR2RGB)

#Vizalizing:
plt.figure(figsize=(40,25))
plt.subplot(131)
plt.imshow(image)
plt.title("Original")
plt.axis('off')

plt.subplot(132)
plt.imshow(mask)
plt.title("Mask")
plt.axis('off')

plt.subplot(133)
plt.imshow(image)
plt.imshow(mask,alpha=.55)
plt.title("Overlap")
plt.axis('off')

In [ ]:
data_path = 'camvid_11/CamVid_11'
print('Number of train frames: ' + str(len(os.listdir(data_path + '/train'))))
print('Number of train labels: ' + str(len(os.listdir(data_path + '/train_labels'))))
print('Number of val frames: ' + str(len(os.listdir(data_path + '/val'))))
print('Number of val labels: ' + str(len(os.listdir(data_path + '/val_labels'))))
print('Number of test frames: ' + str(len(os.listdir(data_path + '/test'))))
print('Number of test labels: ' + str(len(os.listdir(data_path + '/test_labels'))))

In [ ]:
import pandas as pd
classes = pd.read_csv(data_path +'/class_dict.csv', index_col =0)

In [ ]:
classes

In [ ]:
cls2rgb = {cl:list(classes.loc[cl, :]) for cl in classes.index}

In [ ]:
def adjust_mask(mask):

    semantic_map = []
    for colour in list(cls2rgb.values()):
        equality = np.equal(mask, colour)# 256x256x3 with True or False
        class_map = np.all(equality, axis = -1)# 256x256 If all True, then True, else False
        semantic_map.append(class_map)# List of 256x256 arrays, map of True for a given found color at the pixel, and False otherwise.
    semantic_map = np.stack(semantic_map, axis=-1)# 256x256x32 True only at the found color, and all False otherwise.
    return np.float32(semantic_map)# convert to numbers

In [ ]:
idx2rgb={idx:np.array(rgb) for idx, (cl, rgb) in enumerate(cls2rgb.items())}

def map_class_to_rgb(p):

  return idx2rgb[p[0]]


In [ ]:
nm=adjust_mask(mask)
print(nm.shape)

In [ ]:
rgb_mask = np.apply_along_axis(map_class_to_rgb, -1,nm)
print(rgb_mask.shape)

##2.2 Data Augmentation & Loading :
---


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#for training data:

train_image_datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    zoom_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rescale=1./255)
train_mask_datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    zoom_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )

B_size=8

seed = 2
image_generator =train_image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['train'],
    seed=seed,
    batch_size=B_size,
    target_size=(Size,Size))

mask_generator =train_mask_datagen.flow_from_directory(
    data_path,
    classes=['train_labels'],
    class_mode=None,
    seed=seed,
    color_mode='rgb',
    batch_size=B_size,
    target_size=(Size,Size))
# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

def train_generator_fn():
    for (img,mask) in train_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)

In [ ]:
#for validation data:
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator()
val_image_generator = image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['val'],
    seed=seed,
    batch_size=B_size,
    target_size=(Size,Size))

val_mask_generator = mask_datagen.flow_from_directory(
    data_path,
    classes=['val_labels'],
    class_mode=None,
    seed=seed,
    batch_size=B_size,
    color_mode='rgb',
    target_size=(Size,Size))

# combine generators into one which yields image and masks
val_generator = zip(val_image_generator, val_mask_generator)

def val_generator_fn():
    for (img,mask) in val_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)

# 3.1 Create Model:

---



In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import load_img,img_to_array,plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint,EarlyStopping,Callback
from tensorflow.keras.layers import Conv2DTranspose,ZeroPadding2D,Conv2D,add,Dropout,Input,concatenate,MaxPooling2D,UpSampling2D,BatchNormalization,AveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Dropout,LeakyReLU

In [ ]:
#convolutional block with dropout and batch normalization
def double_conv_block(x, kernelsize, filters, dropout, batchnorm=False):
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(x)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(conv)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    return conv

In [ ]:
#downsample_block with batch norm and dropout

def downsample_block(x, n_filters, dropout):
    f = double_conv_block(x, 3, n_filters, dropout=dropout, batchnorm=True)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(dropout)(p)

    return f, p

In [ ]:
#upsample_block with batch norm and dropout
def upsample_block(x, conv_features, n_filters, dropout):
    # upsample
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenate
    x = layers.concatenate([x, conv_features])
    # dropout
    x = layers.Dropout(dropout)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, 3, n_filters, dropout=dropout, batchnorm=True)

    return x

In [ ]:
#gating signal for attention unit
def gatingsignal(input, out_size, batchnorm=False):
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batchnorm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

In [ ]:
from tensorflow.keras import backend as K

#attention unit/block based on soft attention
def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), kernel_initializer='he_normal', padding='same')(x)
    shape_theta_x = K.int_shape(theta_x)
    phi_g = layers.Conv2D(inter_shape, (1, 1), kernel_initializer='he_normal', padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3), strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]), kernel_initializer='he_normal', padding='same')(phi_g)
    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), kernel_initializer='he_normal', padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)
    upsample_psi = layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': shape_x[3]})(upsample_psi)
    y = layers.multiply([upsample_psi, x])
    result = layers.Conv2D(shape_x[3], (1, 1), kernel_initializer='he_normal', padding='same')(y)
    attenblock = layers.BatchNormalization()(result)
    return attenblock

In [ ]:
def Unet(input_size,n_classes):
    """ Input """
    inp = Input(input_size)
    """ Encoder """
    Basemodel= VGG19(include_top=False, weights='imagenet', input_tensor=inp)
    Basemodel.trainable = False

    f1=Basemodel.get_layer("block1_conv2").output
    f2=Basemodel.get_layer("block2_conv2").output
    f3=Basemodel.get_layer("block3_conv4").output
    f4=Basemodel.get_layer("block4_conv4").output

    #"""bridge"""
    """bridge"""
    #bottleneck=PSP(f5)
    bottleneck=Basemodel.get_layer("block5_conv4").output

    """ Decoder """

    # 6 - upsample
    gating_5 = gatingsignal(bottleneck, 512, batchnorm=True)
    att_5 = attention_block(f4, gating_5, 512)
    u6 = upsample_block(bottleneck, att_5, 512, dropout=0.01)
    # 7 - upsample
    gating_7 = gatingsignal(u6, 256, batchnorm=True)
    att_7 = attention_block(f3, gating_7, 256)
    u7 = upsample_block(u6, att_7, 256, dropout=0.01)
    # 8 - upsample
    gating_8 = gatingsignal(u7, 128, batchnorm=True)
    att_8 = attention_block(f2, gating_8, 128)
    u8 = upsample_block(u7, att_8, 128, dropout=0.01)
    # 9 - upsample
    gating_9 = gatingsignal(u8, 64, batchnorm=True)
    att_9 = attention_block(f1, gating_9, 64)
    u9 = upsample_block(u8, att_9, 64, dropout=0.01)


    """ OutPut """
    Output=Conv2D(n_classes, 1, activation = 'softmax')(u9)

    model = Model(inp,Output, name="Unet")
    model.summary()
    return model

In [ ]:
model=Unet((224,224,3),12)

In [ ]:
plot_model(model,show_shapes=True,show_layer_names=True,expand_nested=True)

# 3.2  Callbacks & Trainning:

---


In [ ]:
from keras import backend as K
import numpy as np

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


# Test
y_true = np.array([[0,0,1,0],[0,0,1,0],[0,0,1.,0.]])
y_pred = np.array([[0,0,0.9,0],[0,0,0.1,0],[1,1,0.1,1.]])

r = dice_coef_loss(
    y_true,
    y_pred,
)
print('dice_coef_loss',r)

In [ ]:
import tensorflow as tf
gamma=1.
#alpha=.25
class_weights=[0.125, 0.125, 2.0, 0.125, 0.25, 0.25, 1.0, 1.0, 0.25, 1.0, 1.0, 0.25]
alpha=class_weights
def categorical_focal_loss(y_true, y_pred):
    y_pred /= tf.keras.backend.sum(y_pred, axis=-1, keepdims=True)
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    cross_entropy = -y_true * tf.math.log(y_pred)
    loss = alpha * tf.keras.backend.pow(1 - y_pred, gamma) * cross_entropy
    return tf.keras.backend.sum(loss, axis=1)

In [ ]:
epsilon = 1e-5
smooth = 1

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.4
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

def unified_loss(y_true, y_pred):
  return 0.5*focal_tversky(y_true,y_pred)+0.5*categorical_focal_loss(y_true,y_pred)

In [ ]:
model.compile(optimizer = Adam(learning_rate = 1e-4, weight_decay=5e-5), loss=unified_loss, metrics=['accuracy', tf.keras.metrics.OneHotMeanIoU(num_classes=12)])
#model.compile(optimizer = Adam(learning_rate = 1e-4), loss=dice_coef_loss , metrics=['accuracy', tf.keras.metrics.OneHotMeanIoU(num_classes=12)])
#model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate = 0.1, momentum=0.9, weight_decay=5e-4), loss=unified_loss ,metrics=['accuracy', tf.keras.metrics.OneHotMeanIoU(num_classes=12)])

model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Weights/Camvid_Unet.h5', monitor='val_accuracy',verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,patience=10, min_lr=1e-6)
Earlystop=EarlyStopping(monitor="val_accuracy",patience=20,mode="auto",restore_best_weights=True)

In [ ]:
class Viz_process(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Select a random sample from the validation set
        image_batch, label_batch = next(iter(val_generator))
        idx=np.random.randint(image_batch.shape[0])
        image,label=image_batch[idx],label_batch[idx]
        label=label*1./255
        # Get predicted label
        pred_label = np.squeeze(self.model.predict(np.expand_dims(image, axis=0)),axis=0)
        pred_label=np.argmax(pred_label,axis=-1)
        rgb_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(pred_label,axis=-1))
        rgb_mask=rgb_mask*1./255

        ### Display the image, true label, and predicted label
        plt.figure(figsize=(18,9))
        plt.subplot(131)
        plt.imshow(image)
        plt.axis('off')
        plt.title('Original')
        plt.subplot(132)
        plt.imshow(label)
        plt.axis('off')
        plt.title('Label')
        plt.subplot(133)
        plt.imshow(rgb_mask)
        plt.axis('off')
        plt.title('Pred_Label')
        plt.show()

In [ ]:
epochs=100
steps=int(369/B_size)
valid_steps=int(100/B_size)

Hisotry=model.fit(train_generator_fn(),
        validation_data=val_generator_fn(),
        steps_per_epoch=steps,
        validation_steps=valid_steps,
        epochs=epochs,
        callbacks=[Viz_process(),model_checkpoint,reduce_lr,Earlystop])

# 3.3 Evaluation & Predicting:

---



In [ ]:
#for testing data:
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator()
test_image_generator = image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['test'],
    seed=seed,
    batch_size=B_size,
    target_size=(Size,Size))

test_mask_generator = mask_datagen.flow_from_directory(
    data_path,
    classes=['test_labels'],
    class_mode=None,
    seed=seed,
    batch_size=B_size,
    color_mode='rgb',
    target_size=(Size,Size))

# combine generators into one which yields image and masks
test_generator = zip(test_image_generator, test_mask_generator)

def test_generator_fn():
    for (img,mask) in test_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)

In [ ]:
model.evaluate(test_generator_fn(),steps=len(test_image_generator))

In [ ]:
#predicting:
#image_batch, label_batch = next(iter(test_generator))
image_batch, label_batch = test_image_generator[0], test_mask_generator[0]
idx=np.random.randint(image_batch.shape[0])
idx=0
image,label=image_batch[idx],label_batch[idx]
label=label*1./255
# Get predicted label
pred_label = np.squeeze(model.predict(np.expand_dims(image, axis=0)),axis=0)
pred_label=np.argmax(pred_label,axis=-1)
rgb_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(pred_label,axis=-1))
rgb_mask=rgb_mask*1./255
### Display the image, true label, and predicted label
plt.figure(figsize=(20,9))
plt.subplot(131)
plt.imshow(image)
plt.axis('off')
plt.title('Original')
plt.subplot(132)
plt.imshow(label)
plt.axis('off')
plt.title('Label')
plt.subplot(133)
plt.imshow(rgb_mask)
plt.axis('off')
plt.title('Pred_Label')
plt.show()

In [ ]:
def display_learning_curves(history):
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]

    iou = history.history["one_hot_mean_io_u"]
    val_iou = history.history["val_one_hot_mean_io_u"]

    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs_range = range(100)

    fig = plt.figure(figsize=(12,6))

    plt.subplot(1,2,1)
    plt.plot(epochs_range, iou, label="train iou")
    plt.plot(epochs_range, val_iou, label="validataion iou")
    plt.title("Mean IoU")
    plt.xlabel("Epoch")
    plt.ylabel("Mean IoU")
    plt.legend(loc="lower right")

    plt.subplot(1,2,2)
    plt.plot(epochs_range, loss, label="train loss")
    plt.plot(epochs_range, val_loss, label="validataion loss")
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc="upper right")

    fig.tight_layout()
    plt.show()

In [ ]:
# Display learning curves
display_learning_curves(Hisotry)